## Data exploration and cleaning


### First, I'll load the data and look at the first few entries
* To create the list of types I first ran genfromtxt with *dtype=None* and then manually assigned types and string sizes
* I'm ignoring the first column is this seems to be an index for the week of the year the sample was observed. It should be redundant when we have the date
* I can’t see a reason to include the year field for the same reason
* I imagine the relationship with the date will have a strong non-linear element, i.e. when are avocados in season? I'm going to add another categorical feature for month to try to capture this
* I gather that 4046, 4225, and 4770 are Hass avocado sizes
  * I was thinking about removing the total volume, but the sum of the volumes for the three types does not equal the total
    * I see the same characteristic with the bag size breakdown


In [9]:
import numpy as np

filepath = '/home/ben/Data/morsum/avocado.csv'
types = ["|S10", float, float, float, float, float, float, float, float, float, "|S20", "|S20"]
columnsToIgnore = [0,12]
rawData = np.genfromtxt(filepath, delimiter=',', usecols=np.setdiff1d(range(14), columnsToIgnore), dtype=types, names=True)
rawData[:5]

array([('2015-12-27', 1.33,  64236.62, 1036.74,  54454.85,  48.16, 8696.87, 8603.62,  93.25, 0., 'conventional', 'Albany'),
       ('2015-12-20', 1.35,  54876.98,  674.28,  44638.81,  58.33, 9505.56, 9408.07,  97.49, 0., 'conventional', 'Albany'),
       ('2015-12-13', 0.93, 118220.22,  794.7 , 109149.67, 130.5 , 8145.35, 8042.21, 103.14, 0., 'conventional', 'Albany'),
       ('2015-12-06', 1.08,  78992.15, 1132.  ,  71976.41,  72.58, 5811.16, 5677.4 , 133.76, 0., 'conventional', 'Albany'),
       ('2015-11-29', 1.28,  51039.6 ,  941.48,  43838.39,  75.78, 6183.95, 5986.26, 197.69, 0., 'conventional', 'Albany')],
      dtype=[('Date', 'S10'), ('AveragePrice', '<f8'), ('Total_Volume', '<f8'), ('4046', '<f8'), ('4225', '<f8'), ('4770', '<f8'), ('Total_Bags', '<f8'), ('Small_Bags', '<f8'), ('Large_Bags', '<f8'), ('XLarge_Bags', '<f8'), ('type', 'S20'), ('region', 'S20')])

### Next I need to reformat some of the features for regression
* This mostly means converting categorical features into one-hot vectors and converting the date to epoch time

In [25]:
from datetime import datetime
rawData[0]['Date']
# I would use datetime.timestamp if this was Python 3
epochDT = datetime(1970,1,1)
d = datetime.strptime(rawData[0]['Date'], '%Y-%m-%d')
int((d - epochDT).total_seconds())

1451174400